<a href="https://colab.research.google.com/github/Assault1892/anything-diffusion-on-colab/blob/main/Anything_Diffusion_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

必ず **ドライブのコピー** を押して自分のDriveにコピーしてから作業してください！

In [ ]:
#@title 1. おまじない
import os
os.kill(os.getpid(), 9)

ある種の初期化処理ですがやり方がかなり強引です。  
より良い方法を知っている方がいたら教えてください。多分色々良くないはず。

「セッションがクラッシュした」とか言われますがそれで正常です。次に進んでください。

In [ ]:
#@title 2. Google Driveとの連携 兼 GPU確認
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi

後に使う「config.json」や「出力した画像」を保存するのにマウントが必要です。  
もし `NVIDIA-SMI` と書かれたログが出なかった際は運悪くGPU無しランタイムが割り振られてしまった為，  
上の「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア アクセラレータ」を「GPU」にして一からやり直してください

In [ ]:
#@title 3. Stable Diffusion WebUIをcloneし，設定ファイルを取得
%%bash

git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/webui
cd /content/webui

# config.jsonがDriveに存在するなら取得し，存在しないならgistからAnything Diffusion用configを取得
if [ -e /content/drive/MyDrive/webui-outputs/config.json ]; then
  echo "Config loaded from your drive"
  cp -r /content/drive/MyDrive/webui-outputs/config.json /content/webui/config.json
else
  echo "Config fetching..."
  wget https://gist.githubusercontent.com/Assault1892/50360f8a4103aefe6a1d872c2cc3809a/raw/7cc63bb6c64d36ce9246c3e18b98783295f2c30d/config.json -O /content/webui/config.json
fi

# 同様にui-config.jsonが存在するなら取得し，存在しないならgistから取得
if [ -e /content/drive/MyDrive/webui-outputs/ui-config.json ]; then
  echo "UI Config loaded from your drive"
  cp -r /content/drive/MyDrive/webui-outputs/ui-config.json /content/webui/ui-config.json
else
  echo "UI Config fetching..."
  wget https://gist.githubusercontent.com/Assault1892/55b9ee9040571c68887a1b9dd58511b9/raw/c82bec338d33d4c63ba6c07da135f867bea77f52/ui-config.json -O /content/webui/ui-config.json
fi

# 同様にstyles.csvが存在するなら取得し，存在しないならgistから取得
if [ -e /content/drive/MyDrive/webui-outputs/styles.csv ]; then
  echo "Style file loaded from your drive"
  cp -r /content/drive/MyDrive/webui-outputs/styles.csv /content/webui/styles.csv
else
  echo "Style file fetching..."
  wget https://gist.githubusercontent.com/Assault1892/09944e75ecc7a009194cbcd95716aa51/raw/57e98aa9de188ddcf896210dd130ff42feeffd27/styles.csv -O /content/webui/styles.csv
fi

config.json, ui-config.json, styles.csvをDriveから取得しようとし，存在しないならgistからAnything用に調整したファイルを引っ張ってきます  
だいぶ適当に書いたのでそのうち最適化します

In [ ]:
#@title 4. モデルのCheckpointとVAEを取得し，リネーム
%%bash
cd /content/webui/models/Stable-diffusion/
wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt
wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt
mv Anything-V3.0-pruned.ckpt model.ckpt
mv Anything-V3.0.vae.pt model.vae.pt
ls

モデルファイルをDLしリネーム 大体5分ぐらいはかかります  
もし `IOPub data rate exceeded` と出てもログの中に「model.ckpt」と「model.vae.pt」と書かれた行があればOKです  
あったら次へ 無かったら教えてください 対策を考えます

In [ ]:
#@title 5. 依存関係のインストール
%cd webui
!COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py

ちょっと長いです。

In [ ]:
#@title 6. WebUIの実行
#@markdown **このセルだけは実行が終了しません。** 再生ボタンが回転している状態=正常に動作している状態です。

#@markdown しばらくして出てくる `[https://]xxxxxx(文字列).gradio.app` にアクセスしてください。

#@markdown **出てきてすぐアクセスすると「502 Bad Gateway」が出るので落ち着いて** 多分Gradioが悪い

#@markdown 下のuser, pwには適宜ユーザー名とパスワードを入力してください。gradio.appにアクセスしたときに必要です。

user='me' #@param {type:"string"}
pw='qwerty' #@param {type:"string"}
!COMMANDLINE_ARGS="--share --gradio-debug --gradio-auth {user}:{pw}" REQS_FILE="requirements.txt" python launch.py

「ログイン画面とかいらねーよ」って場合はuserとpwの行を削除し，  
`COMMANDLINE_ARGS` 内の `--gradio-auth {user}:{pw}` を削除してください

飽きたら普通に停止ボタン押しちゃって大丈夫です

In [ ]:
#@title 7. Driveに生成物とコンフィグを保存
%%bash
cp -r "/content/webui/outputs" "/content/drive/MyDrive/webui-outputs/$(TZ=JST-9 date +%Y%m%d%H%M)-outputs"
echo "Outputs exported"
cp -r "/content/webui/config.json" "/content/drive/MyDrive/webui-outputs/"
echo "Config exported"
cp -r "/content/webui/ui-config.json" "/content/drive/MyDrive/webui-outputs"
echo "UI Config exported"
cp -r "/content/webui/styles.csv" "/content/drive/MyDrive/webui-outputs"
echo "Style file exported"

生成物やコンフィグ類が全てDriveの「webui-outputs」フォルダに保存されます  
可能性は薄いとは思いますがファイル名被りによる上書きを防止する為，outputsディレクトリの先頭に日付がつきます